<div class="alert alert-block alert-info" style="border-radius: 20px;">
<i>Hello User.!<br></i> Welcome to <b>python-iLOrest-library</b> <br> This is a example <b>modify_user_account.</b> <br> As you are here let's get it done easily.<br>
    To perform this u need to have <a href = "https://github.com/HewlettPackard/python-ilorest-library" style = "background-color:white">python-ilorest-library</a> installed in your machine.<br>
    And please keep your username and password handy!
    <br>
    <b>Also do execute each successive cell once to avoid any errors!</b>
    
</div>

 <div class="alert alert-block alert-success" style="border-radius: 20px;">
    <h1> 1.</h1>
Below we are importing all the necessary modules from the ilorest library!.<br>
    <li><a href = "https://www.tutorialspoint.com/what-is-python-s-sys-module#:~:text=%20What%20is%20Python%27s%20Sys%20Module%20%201,%5B0%5D%20prints%20the...%205%20Output.%20%20More%20">sys </a>
    </li>
    <li>json </li>
    <li> redfish </li>
    <li>get_resource_directory</li>
    
</div>

In [ ]:
import sys
import json
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError
from get_resource_directory import get_resource_directory

<div class = "alert alert-block alert-success" style="border-radius: 20px;"><h2 style="font-size:40px">2.</h2>This is our modify_ilo_user_account function overview, in the end this will help you to modify the account.<br>
    <i>Let me give you a walk through of it!</i>
    <br>
  There is a hierarchy involved in iLO Rest Api. So we are simply following the hierarchy present to change the settings..

1. In the <b>modify_user_account</b> function we are first using <b>resource_instances</b> variable to obtain information about the resource directory by using get_resource_directory module.
<br>
    
2. Retrieves the account service URI, sends a <b>GET <b/>request to the account service, and obtains the account collection URI. If a role_id is provided, it creates a request body with the modified <b>RoleId</b> parameter.
    <br>
    
3. Else Iterate over resource instances to find the account collection URI. Modify privileges and/or login name if specified for an HPE server.
    <br>
    
4. If new User name is there then modified the new user name and password in the body.
   Iterate over account URIs to find the one with a matching username. If found, modify the account using a <b>PATCH</b> request to the identified URI.
    <br>
    
5.Handling the error after inserting the iso image in slot 
If status code is <b>400</b> the code attempts to retrieve and print extended error message .
If status code is <b> 200 OK</b> then it will print error meassage and status code.
Else the modification is <b>success</b> then code print result in <b>JSON</b> format.



 </div>

In [ ]:
def modify_ilo_user_account(_redfishobj, username_to_modify, new_loginname, new_username, \
                         new_password, role_id, privilege_dict):

    account_collection_uri = None

    resource_instances = get_resource_directory(_redfishobj)

    if DISABLE_RESOURCE_DIR or not resource_instances:
        #resource directory is not available so we will navigate through paths manually to obtain
        #account info
        account_service_uri = _redfishobj.root.obj['AccountService']['@odata.id']
        account_service_response = _redfishobj.get(account_service_uri)
        account_collection_uri = account_service_response.obj['Accounts']['@odata.id']
        #modify role id
        if role_id:
            body = {"RoleId": role_id}
    else:
        #obtain all account instances from resource directory
        for instance in resource_instances:
            if '#ManagerAccountCollection.' in instance['@odata.type']:
                account_collection_uri = instance['@odata.id']
        if privilege_dict:
            #HPE server, so modify privileges
            body = {"Oem": {"Hpe": {"Privileges": {}}}}
            for priv in privilege_dict:
                body["Oem"]["Hpe"]["Privileges"][priv] = privilege_dict[priv]
        if new_loginname:
            #modify login name
            body["Oem"]["Hpe"]["LoginName"] = new_loginname

    if new_username:
        body["UserName"] = new_username
    if new_password:
        body["Password"] = new_password

    #find the account to modify
    account_uri_to_modify = None
    account_uris = REDFISHOBJ.get(account_collection_uri)
    for account_uri in account_uris.dict['Members']:
        account = REDFISHOBJ.get(account_uri['@odata.id'])
        if account.dict['UserName'] == username_to_modify:
            account_uri_to_modify = account_uri['@odata.id']
            break

    if not account_uri_to_modify:
        sys.stderr.write("Cannot find account to modify")
        return

    #modify the account by sending patch request
    resp = REDFISHOBJ.patch(account_uri_to_modify, body)

    #If iLO responds with soemthing outside of 200 or 201 then lets check the iLO extended info
    #error message to see what went wrong
    if resp.status == 400:
        try:
            print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, sort_keys=True))
        except Exception as excp:
            sys.stderr.write("A response error occurred, unable to access iLO Extended Message "\
                             "Info...")
    elif resp.status != 200:
        sys.stderr.write("An http response of '%s' was returned.\n" % resp.status)
    else:
        print("Success!\n")
        print(json.dumps(resp.dict, indent=4, sort_keys=True))

<div class="alert alert-block alert-success" style="border-radius: 20px;"><h2 style="font-size:40px">3.</h2>
<b>The Main function: </b> PLEASE ENTER YOU LOGIN CREDENTIALS HERE TO PROCEED FURTHER BASED WHERE YOU ARE RUUNIG THE SERVER<br>
    <ul>
        <li>The variables <b>USERNAME_TO_MODIFY, NEW_LOGINNAME, NEW_USERNAME, NEW_PASSWORD,</b> and <b>ROLE_ID</b> have been assigned with the specified values. </li>
        <li><b>PRIVILEGE_DICT</b> represents a dictionary where each key corresponds to a specific privilege and the corresponding value indicates whether that privilege is enabled or disabled.</li>
        <li>Using the redfish object we are trying to establish the connection with the server.After establish the connection we try to login .If the server is unreachable we noftify the user with error message.</li>
        <li>If login is successful then we are calling the function with proper parameter. </li>
        <li>After getting required information <b>Redfish.logout()</b> closses the particular session and logs us out of the system.</li>
    </ul>
</div>

In [1]:
if __name__ == "__main__":
    # When running on the server locally use the following commented values
    #SYSTEM_URL = None
    #LOGIN_ACCOUNT = None
    #LOGIN_PASSWORD = None

    # When running remotely connect using the secured (https://) address,
    # account name, and password to send https requests
    # SYSTEM_URL acceptable examples:
    # "https://10.0.0.100"
    # "https://ilo.hostname"
    SYSTEM_URL = "https://10.0.0.100"
    LOGIN_ACCOUNT = "admin"
    LOGIN_PASSWORD = "password"

    #username of the account to modify
    USERNAME_TO_MODIFY = "bruce_wayne"

    #account login name to change the account to
    NEW_LOGINNAME = "joker"

    #account user name to change the account to
    NEW_USERNAME = "joker"

    #account password to change the account to
    NEW_PASSWORD = "joker123"

    #role to change account to
    ROLE_ID = "ReadOnly" #Administrator, ReadOnly or Operator are available

    #update HPE account privileges
    PRIVILEGE_DICT = {"iLOConfigPriv": False, "VirtualMediaPriv": False, "RemoteConsolePriv": True,\
                      "UserConfigPriv": False, "VirtualPowerAndResetPriv": False, \
                      "SystemRecoveryConfigPriv": False, "LoginPriv": True, \
                      "HostStorageConfigPriv": False, "HostNICConfigPriv": False, \
                      "HostBIOSConfigPriv": False}
    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = False

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, \
                                                                            password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    modify_ilo_user_account(REDFISHOBJ, USERNAME_TO_MODIFY, NEW_LOGINNAME, NEW_USERNAME, \
                            NEW_PASSWORD, ROLE_ID, PRIVILEGE_DICT)

    REDFISHOBJ.logout()
 


NameError: name 'ServerDownOrUnreachableError' is not defined

<div class = "alert alert-block alert-warning" style="border-radius: 20px;"><h2 style="font-size:40px">4.</h2>
    <b>When running on the server locally use the following commented values</b><br>
 SYSTEM_URL = None<br>
 LOGIN_ACCOUNT = None<br>
 LOGIN_PASSWORD = None<br>

   <b>When running remotely</b> connect using the secured (https://) address,
     account name, and password to send https requests
    <br>
     SYSTEM_URL acceptable examples:<br>
     https://10.0.0.100<br>
     https://ilo.hostname
</div>

<div class = "alert alert-block alert-success" style="border-radius: 20px;"><h2 style="font-size:40px">5.</h2>In case you need help 🔗 links to relevant resources :
    <br>
    1.Python-ilorest-library:<a href = "https://github.com/HewlettPackard/python-ilorest-library">LINK</a><br>
    2.Hpe ilorest-api explorer:<a href = "https://ilorestfulapiexplorer.ext.hpe.com/">LINK</a><br>
    3.Rest api: <a href = "https://restfulapi.net/">LINK</a><br>
    
</div>